# This notebook assumes that you have extracted the embeddings (using the procedure mentioned in the 0_ESM_Embeddings_Extractor.ipynb notebook) and have stored them in a zipped format

## Since we have used google colab; we copy the embeddings from google drive before training the model; similar procedure can be used to run it locally;

# for the external validation on pmtNET we use the beta-peptide pairs dataset from MCPAS and VDJDB

## A example is shown below for CDR3beta and peptide: for MCPAS

In [ ]:
#####################################################1_origMCPAS_NoMHC/train_B
!cp -r /content/drive/MyDrive/TCR-pMHC-results/mcpas/1_origMCPAS_NoMHC/train_B/*.csv '/content/'
!cp -r /content/drive/MyDrive/TCR-pMHC-results/mcpas/1_origMCPAS_NoMHC/train_B/*.zip '/content'

## A example is shown below for CDR3beta, peptide: for VDJDB

In [ ]:
                                                 ##########################################################
# ###################################################3_origMCPAS_HMHC/train_AB
!cp -r /content/drive/MyDrive/TCR-pMHC-results/mcpas/3_origMCPAS_HMHC/train_B/*.csv '/content/'
!cp -r /content/drive/MyDrive/TCR-pMHC-results/mcpas/3_origMCPAS_HMHC/train_B/*.zip '/content'                                         

## Example of pmtnet data External Validation

In [1]:
!cp -r /content/drive/MyDrive/TCR-pMHC-results/pmtnet_exp/set1/*.csv '/content/'
!cp -r /content/drive/MyDrive/TCR-pMHC-results/pmtnet_exp/set1/*.zip '/content'

## Unzip the embeddings to folder for developing the train and test set (pmtNET)

In [ ]:
########################################################################## AB and A
#### unzip train
!unzip -q <path_to_train_cdr3b.zip>     -d  train_cdr3b
!unzip -q <path_to_train_peptide.zip>   -d  train_peptide

#### unzip test
!unzip -q <path_to_test_cdr3b.zip>     -d  test_cdr3b
!unzip -q <path_to_test_peptide.zip>   -d  test_peptide

In [ ]:
import sys
import numpy as np
import pandas as pd
import os
from tqdm import tqdm
from natsort import natsorted
import matplotlib.pyplot as plt
plt.style.use('seaborn')
%matplotlib inline
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Layer,Input, Dense, Dropout, Activation, Concatenate, Flatten, BatchNormalization
from tensorflow.keras.regularizers import l2,l1
from tensorflow.keras.optimizers import SGD,Adam,RMSprop
import tensorflow.keras.backend as K
from tensorflow.keras.preprocessing.image import array_to_img, img_to_array, load_img
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint,ReduceLROnPlateau
from tensorflow.keras.models import load_model
import tensorflow.keras.metrics
####import tensorflow_addons as tfa not required
from sklearn.preprocessing import LabelEncoder
import tensorflow as tf
tf.random.set_seed(1)
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.utils import to_categorical, plot_model
import sklearn
import os
from natsort import natsorted
from sklearn.metrics import *
import torch
from tqdm import tqdm

## For MCPAS and VDJDB with beta-peptide MHC information

### Model CDR3b + peptide

In [ ]:
### train files
path_train_cdr3b = <'path_to_train_cdr3b_embeddings'>
path_train_pepti = <'path_to_train_peptide_embeddings'> 

### test files
path_test_cdr3b = <'path_to_test_cdr3b_embeddings'>  
path_test_pepti = <'path_to_test_peptide_embeddings'>  


trainmat_cdr3b = os.listdir(path_train_cdr3b) 
trainmat_pepti = os.listdir(path_train_pepti)


testmat_cdr3b = os.listdir(path_test_cdr3b) 
testmat_pepti = os.listdir(path_test_pepti)

## natsort is used to order the pairs as they appear in the .csv; this would be helpful later to map the pairs with their respective labels

In [ ]:
### AB-pep, A-pep, B-pep

###train
train_nmat_cdr3b = natsorted(trainmat_cdr3b)
train_nmat_pepti = natsorted(trainmat_pepti)



###test
test_nmat_cdr3b = natsorted(testmat_cdr3b)
test_nmat_pepti = natsorted(testmat_pepti)

## the following step extracts the embeddings and stores in a numpy matrix for MCPAS subset 1


In [ ]:
train_matmat_cdr3b = np.zeros((train_samples,1280))
train_matmat_pepti = np.zeros((train_samples,1280))


test_matmat_cdr3b = np.zeros((test_samples,1280))
test_matmat_pepti = np.zeros((test_samples,1280))

### load train samples 

for i in tqdm(range(train_samples)):

    train_matmat_cdr3b[i] = torch.load(path_train_cdr3b+train_nmat_cdr3b[i])['mean_representations'][33]

    train_matmat_pepti[i] = torch.load(path_train_pepti+train_nmat_pepti[i])['mean_representations'][33]


    for j in tqdm(range(test_samples)):

    test_matmat_cdr3b[j]  = torch.load(path_test_cdr3b+test_nmat_cdr3b[j])['mean_representations'][33]

    test_matmat_pepti[j]  = torch.load(path_test_pepti+test_nmat_pepti[j])['mean_representations'][33]



## load *.csv files for labels

### for mcpas subset 1

In [ ]:
df_train = pd.read_csv('<path-to-mcpas/vdjdb-train_B.csv>')
df_test  = pd.read_csv('<path-to-pmtNet-set>')

In [ ]:
############ load labels
y_train = df_train['sign'].values.reshape(-1,1)
y_test  = df_test['sign'].values.reshape(-1,1)

In [ ]:
### model
def clear_sess():
  try:
    del model 
    del history 
  except:
    pass
  from tensorflow.keras import backend as K
  K.clear_session()
  import gc
  gc.collect()



  return None

def keras_mcc(y_true, y_pred):
    tp = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    tn = K.sum(K.round(K.clip((1 - y_true) * (1 - y_pred), 0, 1)))
    fp = K.sum(K.round(K.clip((1 - y_true) * y_pred, 0, 1)))
    fn = K.sum(K.round(K.clip(y_true * (1 - y_pred), 0, 1)))

    num = tp * tn - fp * fn
    den = (tp + fp) * (tp + fn) * (tn + fp) * (tn + fn)
    return num / K.sqrt(den + K.epsilon())

### CDR3B peptide 

In [ ]:
clear_sess()

acti  = 'swish'

#input_1


input_1 = Input(shape = (1280,), name='i_1')
dense1_1 = Dense(128, activation = acti)(input_1)
bn1_1 = BatchNormalization()(dense1_1)
drop_1 = Dropout(0.5)(bn1_1)

#input_2
input_2 = Input(shape = (1280,), name='i_2')
dense2_1 = Dense(128, activation = acti)(input_2)
bn2_1 = BatchNormalization()(dense2_1)
drop_2 = Dropout(0.5)(bn2_1)


# concatenate
concat   = Concatenate()([drop_1,drop_2])
fc_1   = Dense(512, activation = acti)(concat)
drop_4 = Dropout(0.5)(fc_1)
fc_2   = Dense(256, activation = acti)(drop_4)
#classification output- TCR-Peptide Binding yes/no
output  = Dense(1, activation = 'sigmoid')(fc_2)
 
# create model with two inputs
model = Model(inputs=[input_1,input_2], outputs=output)

In [ ]:
metrics_c = [tensorflow.keras.metrics.AUC(name="auc_roc",curve="ROC"),tensorflow.keras.metrics.AUC(name="auc_pr",curve="PR"),keras_mcc]
model.compile(loss='binary_crossentropy', optimizer=tensorflow.keras.optimizers.Adam(learning_rate=0.009),   metrics=metrics_c)
reduce_lr = ReduceLROnPlateau(monitor='val_keras_mcc', factor=0.95,patience=50, min_lr=0.005, verbose=0)

In [ ]:
checkpoint_filepath_1 = 'weights-improvement-val-auc-pr.hdf5'
model_checkpoint_callback_1 = ModelCheckpoint(filepath=checkpoint_filepath_1,save_weights_only=False,monitor='val_auc_pr',mode='max',save_best_only=True)

checkpoint_filepath_2 = 'weights-improvement-val-keras-mcc.hdf5'
model_checkpoint_callback_2 = ModelCheckpoint(filepath=checkpoint_filepath_2,save_weights_only=False,monitor='val_keras_mcc',mode='max',save_best_only=True)

### fit the keras model for CDR3B and peptide

In [ ]:
history=model.fit([train_matmat_cdr3b,train_matmat_pepti],y_train,
                  batch_size=1024, epochs=1000,
                  validation_split=0.1,
                  callbacks=[model_checkpoint_callback_1, model_checkpoint_callback_2,reduce_lr])

## once the best model is trained, we can test it over the evaluation dataset

In [ ]:
model_loaded = '/content/weights-improvement-val-keras-mcc.hdf5'
model = tensorflow.keras.models.load_model(model_loaded,compile=False, custom_objects={'metrics_c': keras_mcc})

y_pred = model.predict([test_matmat_cdr3b, test_matmat_pepti])
y_act = y_test.flatten()
y_pred= y_pred.flatten()
y_pred_c=np.where(y_pred>0.5,1,0)


print(roc_auc_score(y_act, y_pred),average_precision_score(y_act, y_pred), matthews_corrcoef(y_act,y_pred_c),f1_score(y_act,y_pred_c))

In [ ]:
####model load max aucpr
model_loaded = '/content/weights-improvement-val-auc-pr.hdf5'
model = tensorflow.keras.models.load_model(model_loaded,compile=False)

y_pred = predict_mode(mode, test_matmat_cdr3a, test_matmat_cdr3b, test_matmat_pepti, test_matmat_mhc, model_loaded, model)

y_act = y_test.flatten()
y_pred= y_pred.flatten()
y_pred_c=np.where(y_pred>0.5,1,0)


print(roc_auc_score(y_act, y_pred),average_precision_score(y_act, y_pred),matthews_corrcoef(y_act,y_pred_c),cohen_kappa_score(y_act,y_pred_c))

# Not in train + 0.8 and 0.9 sequence similarity


In [ ]:
from Bio import pairwise2

In [ ]:
tempdf_train = df_train.copy()
tempdf_test = df_test.copy()

In [ ]:
def calculate_similarity(seq1, seq2):
    # Compute the local alignment score between two sequences
    alignments = pairwise2.align.localxx(seq1, seq2)
    # Check if alignments list is empty
    if not alignments:
        return 0.0
    # Extract the alignment with the highest score
    best_alignment = max(alignments, key=lambda x: x[2])
    # Calculate the similarity as the alignment score divided by the length of the longer sequence
    similarity = best_alignment[2] / max(len(seq1), len(seq2))
    return similarity

In [ ]:
unique_train_peptides = tempdf_train['peptide'].unique()
unique_test_peptides = tempdf_test['Antigen'].unique()

In [ ]:
mat = np.zeros((len(unique_test_peptides),len(unique_train_peptides)))

for test_index,test_peptide in tqdm(enumerate(unique_test_peptides), desc="Calculating similarities"):
    similarities = []
    for train_index,train_peptide in enumerate(unique_train_peptides):
        similarity = calculate_similarity(test_peptide, train_peptide)
        mat[test_index][train_index] = similarity
        # print(similarity)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

plt.imshow(mat)
plt.colorbar()
plt.show()

In [ ]:
max_in_rows = np.max(mat, axis=1)

In [ ]:
pepsim_dict = dict(zip(unique_test_peptides, max_in_rows))

In [ ]:
tempdf_test['sim'] = tempdf_test['Antigen'].map(pepsim_dict)

## Result 1: test predictions

In [ ]:
# subset0
print('Test Predict:', matthews_corrcoef(y_act,y_pred_c), roc_auc_score(y_act, y_pred),average_precision_score(y_act, y_pred),f1_score(y_act,y_pred_c))

## Result 2: test peptides: Not in train

In [ ]:
y_act_nit = y_act[~tempdf_test['Antigen'].isin(unique_train_peptides)]
y_pred_nit= y_pred[~tempdf_test['Antigen'].isin(unique_train_peptides)]
y_pred_c_nit=y_pred_c[~tempdf_test['Antigen'].isin(unique_train_peptides)]

print('Test Predict not in Train:', matthews_corrcoef(y_act_nit,y_pred_c_nit), roc_auc_score(y_act_nit, y_pred_nit),average_precision_score(y_act_nit, y_pred_nit),f1_score(y_act_nit,y_pred_c_nit))

## Result 3: test peptides: not in train and max 90% similarity cutoff

In [ ]:
y_act_09 = y_act[tempdf_test['sim'] <= 0.9]
y_pred_09= y_pred[tempdf_test['sim'] <= 0.9]
y_pred_c_09=y_pred_c[tempdf_test['sim'] <= 0.9]

print('0.9 Similarity', matthews_corrcoef(y_act_09,y_pred_c_09), roc_auc_score(y_act_09, y_pred_09),average_precision_score(y_act_09, y_pred_09),f1_score(y_act_09,y_pred_c_09))

## Result 4: test peptides: not in train and max 80% similarity cutoff

In [ ]:
y_act_08 = y_act[tempdf_test['sim'] <= 0.8]
y_pred_08= y_pred[tempdf_test['sim'] <= 0.8]
y_pred_c_08=y_pred_c[tempdf_test['sim'] <= 0.8]

print('0.8 Similarity:', matthews_corrcoef(y_act_08,y_pred_c_08), roc_auc_score(y_act_08, y_pred_08),average_precision_score(y_act_08, y_pred_08),f1_score(y_act_08,y_pred_c_08))